In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from ast import literal_eval

from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

We load in our datasets, consisting of user data and data for the recipes. We will be using the ratings that users gave in combination with the recipe's ingredients, techniques and calories to perform recommendations

In [40]:
df_users = pd.read_csv('Data/PP_users.csv')
#Note that there are no duplicates
df_users.head()

# u = user_id, 
# techniques = techniques used for items  that were interacted with(index is a technique with the number being a counter),
# items = item_ids of items that were interacted with, 
# n_items = number of items reviewed, 
# ratings = ratings for items reviewed, 
# n_ratings = number of ratings

number_of_users_with_less_than_10_reviews = df_users[df_users["n_ratings"] < 10].shape[0]
number_of_users_with_less_than_5_reviews = df_users[df_users["n_ratings"] < 5].shape[0]
number_of_users = len(df_users.index)

print(str(round((number_of_users_with_less_than_10_reviews / number_of_users) * 100,
                1)) + "% of user have less than 10 reviews")
print(str(round((number_of_users_with_less_than_5_reviews / number_of_users) * 100,
                1)) + "% of user have less than 5 reviews")


62.9% of user have less than 10 reviews
39.9% of user have less than 5 reviews


In [41]:
df_recipes = pd.read_csv('Data/PP_recipes.csv')
#Note that there are no duplicates
df_recipes.head()

# id = recipe_id, i = Recipe ID mapped to contiguous integers from 0, 
# name_tokes = BPE-tokenized recipe name,
# ingredient_tokens = BPE-tokenized ingredients list (list of lists), 
# steps_tokens = BPE-tokenized steps, 
# techniques = List of techniques used in recipe,
# calorie_level = either a 0, 1 or 2 indicating how much calories it contains,
# ingredient_ids = the ids of the ingredients used

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [42]:
df_users = pd.read_csv('Data/PP_users.csv')
df_users.drop('techniques', axis=1, inplace=True)
df_users.drop('n_items', axis=1, inplace=True)
df_users.drop('n_ratings', axis=1, inplace=True)

df_users = df_users.rename(columns={'u': 'user', 'items': 'item', 'ratings': 'rating'})

df_users.head()

,user,item,rating
0,0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1,1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
3,3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
4,4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."


In [43]:
# Needed to make the explode function, source: https://stackoverflow.com/questions/63472664/pandas-explode-function-not-working-for-list-of-string-column
df_users['rating'] = df_users['rating'].apply(literal_eval)
df_users['item'] = df_users['item'].apply(literal_eval)

df_users = df_users.explode(['rating', 'item'], ignore_index=True)
df_users.head()

,user,item,rating
0,0,1118,5.0
1,0,27680,5.0
2,0,32541,5.0
3,0,137353,5.0
4,0,16428,5.0


In [44]:
df_recipes.drop('i', axis=1, inplace=True)
df_recipes.drop('name_tokens', axis=1, inplace=True)
df_recipes.drop('ingredient_tokens', axis=1, inplace=True)
df_recipes.drop('steps_tokens', axis=1, inplace=True)

df_recipes.head()

,id,techniques,calorie_level,ingredient_ids
0,424415,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [45]:
# Needed to make the explode function, source: https://stackoverflow.com/questions/63472664/pandas-explode-function-not-working-for-list-of-string-column
df_recipes['techniques'] = df_recipes['techniques'].apply(literal_eval)
df_recipes['ingredient_ids'] = df_recipes['ingredient_ids'].apply(literal_eval)

df_recipes = df_recipes.explode('techniques')
df_recipes = df_recipes.explode('ingredient_ids')
df_recipes.head()

,id,techniques,calorie_level,ingredient_ids
0,424415,0,0,389
0,424415,0,0,7655
0,424415,0,0,6270
0,424415,0,0,1527
0,424415,0,0,3406


TODO: use algorithm, verify results

## Health Filter

In [46]:
# Note that this file is made by us to only have to load nutritional info instead of a giant csv file
nutrition_data = pd.read_csv('Data/nutrition.csv', sep=';')
nutrition_data.head()

,id,nutrition
0,38,"[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]"
1,39,"[1110.7, 90.0, 81.0, 15.0, 126.0, 82.0, 28.0]"
2,40,"[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]"
3,41,"[536.1, 36.0, 128.0, 64.0, 58.0, 19.0, 21.0]"
4,43,"[437.9, 29.0, 170.0, 11.0, 13.0, 54.0, 19.0]"


In [47]:
# This functions returns the recipes 
# Note: limits takes max sugar amount, max sodium, min protein amount, max saturated_fat
def ApplyHealthFilter(recipe_ids, limits=[.15, .35, .10, .25], debug_prints=False):
    healthy_recipes = []

    # Get the nutritional information for the relevant recipes
    recipes = nutrition_data.loc[nutrition_data['id'].isin(recipe_ids)]

    for index, recipe in recipes.iterrows():
        # Nutrition information in calories, total fat, sugar, sodium, protein, saturated fat, carbohydrates 
        nutrition_values = recipe['nutrition']
        # convert string version of array into a proper array
        nutrition_values = literal_eval(nutrition_values)

        sugar = nutrition_values[2]
        sodium = nutrition_values[3]
        protein = nutrition_values[4]
        saturated_fat = nutrition_values[5]

        # Since the nutritional info is in absolute numbers instead of per 100 grams, we'll normalize
        normalization_factor = sum(nutrition_values[1:])
        normalization_factor = max(normalization_factor, 0.01)

        sugar /= normalization_factor
        sodium /= normalization_factor
        protein /= normalization_factor
        saturated_fat /= normalization_factor

        if sugar < limits[0] and sodium < limits[1] and protein > limits[2] and saturated_fat < limits[3]:
            healthy_recipes.append(recipe['id'])

            if debug_prints:
                print("Healthy: ", index)

        else:
            if debug_prints:
                print("Unhealthy: ", index, sugar, sodium, protein, saturated_fat)

    return healthy_recipes


df_random = df_users.sample(n=20)
recipe_ids = df_random['item']
# limits = [.15, .35, .10, .25]

healhty_recipes = ApplyHealthFilter(recipe_ids)
print(healhty_recipes)

[53586, 84733, 93179, 141621, 159337]


## 1. Individual Recommendations

In [48]:
df_random = df_users.sample(n=10000)

In [49]:
print(df_users)
print(df_recipes)

X = df_random.drop('rating', axis=1)  # training data
y = df_random['rating']  # target values
# TODO take into account df_recipes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)  # 67% training, 33% testing

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred_train = knn.predict(X_train)  # y_pred_train predicts X_train
y_pred_test = knn.predict(X_test)

#y_pred_train = np.rint(y_pred_train)
#y_pred_test = np.rint(y_pred_test)

#print(X_train)
#print((y_pred_train))
#print((np.array(y_train.to_list())))

# TODO compare expected outputs with actual outputs
print(y_pred_test)  # why float value ??
print(y_test)

# TODO solve
print('Accuracy on training data =', metrics.accuracy_score(np.array(y_train.to_list()), y_pred_train))
print('Accuracy on testing data =', metrics.accuracy_score(np.array(y_test.to_list()), y_pred_test))
print('')
print(metrics.classification_report(np.array(y_test.to_list()), y_pred_test))

         user    item rating
0           0    1118    5.0
1           0   27680    5.0
2           0   32541    5.0
3           0  137353    5.0
4           0   16428    5.0
...       ...     ...    ...
698896  25073  172034    5.0
698897  25074  157808    5.0
698898  25074  159896    5.0
698899  25075  166686    4.0
698900  25075  157084    4.0

[698901 rows x 3 columns]
            id techniques  calorie_level ingredient_ids
0       424415          0              0            389
0       424415          0              0           7655
0       424415          0              0           6270
0       424415          0              0           1527
0       424415          0              0           3406
...        ...        ...            ...            ...
178264  263840          0              0            335
178264  263840          0              0           1563
178264  263840          0              0           1511
178264  263840          0              0           3248
178264  2

ValueError: Classification metrics can't handle a mix of multiclass and unknown targets

In [ ]:
df_users['rating'].value_counts()

In [50]:
df_recipes_full = pd.read_csv('Data/RAW_recipes.csv')
#Note that there are no duplicates
df_recipes_full.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [51]:
df_recipes_full.drop('minutes', axis=1, inplace=True)
df_recipes_full.drop('contributor_id', axis=1, inplace=True)
df_recipes_full.drop('submitted', axis=1, inplace=True)
df_recipes_full.drop('tags', axis=1, inplace=True)
df_recipes_full.drop('nutrition', axis=1, inplace=True)
df_recipes_full.drop('steps', axis=1, inplace=True)
df_recipes_full.drop('ingredients', axis=1, inplace=True)
df_recipes_full.drop('description', axis=1, inplace=True)

df_recipes_full = df_recipes_full.rename(columns={'id': 'item'})

df_recipes_full.head()

,name,item,n_steps,n_ingredients
0,arriba baked winter squash mexican style,137739,11,7
1,a bit different breakfast pizza,31490,9,6
2,all in the kitchen chili,112140,6,13
3,alouette potatoes,59389,11,11
4,amish tomato ketchup for canning,44061,5,8


In [52]:
users_ratings = df_users.groupby(['user']).count()  # count the ratings for each user
selected = users_ratings['rating'] > 30  # keep only 30 + ratings
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample(n=10)

select_column_df = random_selected.reset_index()[
    'user']  # reset_index() create a new index, and the userId became a column. Then, we can filter using the column name
group_users = list(
    select_column_df)  # iloc select by index, since our dataframe only has one row we read it from the index 0
print(group_users)

[1467, 1317, 7098, 1544, 7438, 2178, 1668, 1610, 7095, 4849]


In [53]:
group_ratings = df_users.loc[df_users['user'].isin(group_users)]
total_recipes = set(df_recipes.index.tolist())
num_ratings_df = df_users.groupby(['item']).count()
considered_recipes = set(num_ratings_df.loc[num_ratings_df['user'] >= 30].reset_index()['item'])

group_seen_recipes = set(group_ratings['item'].tolist())
group_unseen_recipes = considered_recipes - group_seen_recipes

print('Total amount of recipes,', len(total_recipes))
print('Recipes that have at least 20 ratings,', len(considered_recipes))
print('Recipes that have been rated by the currently selected group,', len(group_seen_recipes))
print('New recipes that the group didnt try yet,', len(group_unseen_recipes))

Total amount of recipes, 178265
Recipes that have at least 20 ratings, 2456
Recipes that have been rated by the currently selected group, 1279
New recipes that the group didnt try yet, 2167


In [54]:
from IPython.core.display import display
import itertools
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (12) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_users)
group_unseen_df = pd.DataFrame(list(itertools.product(group_users, group_unseen_recipes)), columns=['user', 'item'])
group_unseen_df['predicted_rating'] = recsys.predict(group_unseen_df)
group_unseen_df = group_unseen_df[
    group_unseen_df['predicted_rating'].notna()]  # remove the recipes we couldn't get a prediction for
display(group_unseen_df.head(10))

,user,item,predicted_rating
5,1467,131089,4.577501
20,1467,106555,4.459102
31,1467,155731,4.144755
58,1467,147642,4.237022
85,1467,284,4.363167
107,1467,172390,4.209210
126,1467,106910,4.002193
175,1467,131599,3.736137
210,1467,74368,4.313530
237,1467,139982,3.516179


In [55]:
#Min-Max normalization of predicted_ratings

maxVal = group_unseen_df['predicted_rating'].max()
minVal = group_unseen_df['predicted_rating'].min()
group_unseen_df['predicted_rating'] = (group_unseen_df['predicted_rating'] - minVal) / (
        maxVal - minVal)  # Normalized to 0 - 1 scale
group_unseen_df['predicted_rating'] *= 5  # Normalized to 0 - 5 scale

display(group_unseen_df.head(10))

,user,item,predicted_rating
5,1467,131089,3.623774
20,1467,106555,3.511631
31,1467,155731,3.213895
58,1467,147642,3.301286
85,1467,284,3.420766
107,1467,172390,3.274944
126,1467,106910,3.078867
175,1467,131599,2.826870
210,1467,74368,3.373752
237,1467,139982,2.618535


#### Least Misery strategy

In [56]:
least_misery_df = group_unseen_df.groupby(['item']).min().reset_index()
# TODO: Find name of recipe from the RAW data
least_misery_df = least_misery_df.join(df_recipes_full['name'], on='item')
items_lm = least_misery_df['item'].copy()
healthy_lm = ApplyHealthFilter(items_lm)

least_misery_df['healthy'] = least_misery_df['item'].apply(lambda x: 1 if x in healthy_lm else 0)
least_misery_df = least_misery_df.sort_values(by="predicted_rating", ascending=False)[
    ['item', 'predicted_rating', 'name', 'healthy']]

least_misery_df = least_misery_df[least_misery_df.healthy == 1]
display(least_misery_df.head(10))

,item,predicted_rating,name,healthy
202,31148,4.643931,buffalo chicken flatbread pizza,1
817,121509,4.297019,lemon date bubble bars,1
156,25058,4.293552,blended fruit chiller,1
408,59987,4.292301,crabby cheese angel hair pasta,1
437,65619,4.167429,crock pot chicken tacos,1
424,62689,4.141112,creamy chicken taco soup,1
705,103767,4.123346,healthy salmon,1
1089,169138,4.116159,quick n easy curried shrimp,1
377,54986,4.107593,claudi sanders refrigerator rolls,1
653,97240,4.014044,green beans in beer sauce,1


#### Most Pleasure strategy

In [57]:
most_pleasure_df = group_unseen_df.groupby(['item']).max().reset_index()
# TODO: Find name of recipe from the RAW data
items_mp = most_pleasure_df['item'].copy()
healthy_mp = ApplyHealthFilter(items_mp)

most_pleasure_df['healthy'] = most_pleasure_df['item'].apply(lambda x: 1 if x in healthy_mp else 0)

most_pleasure_df = most_pleasure_df.join(df_recipes_full['name'], on='item').reset_index()
most_pleasure_df = most_pleasure_df.sort_values(by="predicted_rating", ascending=False).reset_index()[
    ['item', 'predicted_rating', 'name', 'healthy']]

most_pleasure_df = most_pleasure_df[most_pleasure_df.healthy == 1]

display(most_pleasure_df.head(10))

,item,predicted_rating,name,healthy
6,31148,4.643931,buffalo chicken flatbread pizza,1
8,172997,4.608767,restaurant style mexican rice,1
12,163739,4.562899,pork tenderloin with balsamic vinegar,1
24,113698,4.501929,jalapeo cheddar potatoes,1
25,177020,4.491375,rootin tootin roasted roots roasted root v...,1
32,36503,4.461164,carne en su jugo estilo tapatio,1
51,81311,4.405517,exotic fruit salad,1
62,150138,4.377502,oriental summer rice salad,1
66,164559,4.372523,potato and sausage in gravy,1
79,54986,4.346351,claudi sanders refrigerator rolls,1


#### Approval Voting

In [60]:
group_unseen_temp_df = group_unseen_df.copy()
group_unseen_temp_df['voted'] = group_unseen_temp_df['predicted_rating'].apply(lambda x: 1 if x > 3.5 else 0)
approval_df = group_unseen_temp_df.groupby(['item']).sum()
approval_df.drop('user', axis=1, inplace=True)
approval_df['predicted_rating'] /= len(random_selected)  # Normalize rating
# Only keep the items with maximum approval
approval_df = approval_df[approval_df.voted == approval_df.voted.max()]
approval_df = approval_df.sort_values(by="predicted_rating",
                                      ascending=False).reset_index()  # Get the best rated items with max approval
approval_df = approval_df.join(df_recipes_full['name'], on='item')
display(approval_df.head(10))

,item,predicted_rating,voted,name
0,11325,3.891502,9,avocado vinaigrette
1,129659,3.493177,9,mango salad with grilled shrimp
